# Maven Power Outage Challenge

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium 
import openpyxl

## Step 0. Initial exploration of the data - it is messy!

In [79]:
# Define the sheet names in a list so that we can easily iterate through the excel sheets (tabs)
sheet_names = []
for n in range(2002, 2024, 1):
    sheet_names.append(str(n))

In [124]:
# Creating a dictionary of dataframes, one dataframe per excel sheet. 
# this will allow us to tailor our data cleaning to the individual sheet since the format of the sheet changes over the years.

DataFrame_dict = {}


for sheet in sheet_names:
    if sheet in ["2002", "2008"]:
        header_value = 2
    else:
        header_value = 1
    
    DataFrame_dict["sheet{0}".format(sheet)] = pd.read_excel('DOE_Electric_Disturbance_Events.xlsx', engine='openpyxl', header=header_value, sheet_name=sheet)

In [125]:
# Check out one of the dataframes to see if we get the data we expect:
DataFrame_dict["sheet2008"]

,Date,NERC Region,Time,Area Affected,Type of Disturbance,Loss (megawatts),Number of Customers Affected 1,Restoration,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Date/Time,NaN,NaN,NaN,NaN
1,January,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2008-01-04 00:00:00,WECC,4:00 a.m.,Northern California,Winter Storm,500,2606931,5:00 p.m. January 14,NaN,NaN,NaN,NaN
3,2008-01-04 00:00:00,WECC,7:47 a.m.,Sacramento County,Severe Storm,300,150000,4:30 p.m. January 04,NaN,NaN,NaN,NaN
4,2008-01-29 00:00:00,WECC,5:00 a.m.,"San Francisco Bay Area, California",Exciter Faulted,NaN,-,12:17 p.m. January 29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
185,2008-12-30 00:00:00,RFC,4:02 p.m.,"Crawfordsville, Indiana",Shed Firm Load,41,9700,4:37 p.m. December 30,NaN,NaN,NaN,NaN
186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
187,[1] Estimated values.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,Note: Estimates for 2008 are preliminary.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
# We need to figure out how to remove rows from the dataframe that are redundant or that repeat the column names.
# To do this I will set the dataframe index to column one which for most of the data will be a date. 
# Lets use DataFrame_dict["sheet2008"] as an example

DataFrame_dict["sheet2008"].set_index("Date")
DataFrame_dict["sheet2008"]["Date"].head(20)

0                     NaN
1               January  
2     2008-01-04 00:00:00
3     2008-01-04 00:00:00
4     2008-01-29 00:00:00
5     2008-01-29 00:00:00
6     2008-01-29 00:00:00
7     2008-01-29 00:00:00
8     2008-01-30 00:00:00
9               February 
10    2008-02-01 00:00:00
11    2008-02-02 00:00:00
12    2008-02-05 00:00:00
13    2008-02-06 00:00:00
14    2008-02-09 00:00:00
15    2008-02-10 00:00:00
16    2008-02-10 00:00:00
17    2008-02-10 00:00:00
18    2008-02-10 00:00:00
19    2008-02-10 00:00:00
Name: Date, dtype: object

In [136]:
# Lets create a function that takes a dataframe, iterates through the rows, and checkes that the "Date" column to a 
# datetime datatype. otherwise it drops the row because this indicates the row is an unexpected format and most likely a human readable marker.

from datetime import datetime

def drop_non_datetime_values(dataframe_name, column_name):
    for ind, row in dataframe_name.iterrows():
        if type(dataframe_name[column_name][ind]) == datetime:
            pass
        else:
            dataframe_name.drop(axis=0, index=ind, inplace=True)
    return dataframe_name

In [140]:
# Testing the function above:

drop_non_datetime_values(DataFrame_dict["sheet2008"], "Date")
DataFrame_dict["sheet2008"]

,Date,NERC Region,Time,Area Affected,Type of Disturbance,Loss (megawatts),Number of Customers Affected 1,Restoration,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
2,2008-01-04 00:00:00,WECC,4:00 a.m.,Northern California,Winter Storm,500,2606931,5:00 p.m. January 14,NaN,NaN,NaN,NaN
3,2008-01-04 00:00:00,WECC,7:47 a.m.,Sacramento County,Severe Storm,300,150000,4:30 p.m. January 04,NaN,NaN,NaN,NaN
4,2008-01-29 00:00:00,WECC,5:00 a.m.,"San Francisco Bay Area, California",Exciter Faulted,NaN,-,12:17 p.m. January 29,NaN,NaN,NaN,NaN
5,2008-01-29 00:00:00,SERC,4:00 p.m.,"Arkansas, Mississippi, North Louisiana",Severe Thunderstorms,NaN,110000,8:00 a.m February 03,NaN,NaN,NaN,NaN
6,2008-01-29 00:00:00,RFC,10:00 p.m.,Southeastern Michigan,Wind/Ice Storm,NaN,86915,6:30 p.m. February 01,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
181,2008-12-26 00:00:00,HI,6:13 p.m.,"Island of Oahu, Hawaii",Lightning,1060,294000,5:00 p.m. December 27,NaN,NaN,NaN,NaN
182,2008-12-27 00:00:00,RFC,4:00 p.m.,Southeastern Michigan,Wind Storm,NaN,247847,11:30 p.m. January 01,NaN,NaN,NaN,NaN
183,2008-12-28 00:00:00,RFC,4:45 a.m.,Michigan Lower Peninsula,Wind Storm,NaN,210517,6:00 p.m. December 31,NaN,NaN,NaN,NaN
184,2008-12-28 00:00:00,RFC,11:45 a.m.,Michigan Lower Peninsula,Wind Storm,NaN,230000,11:30 p.m. December 28,NaN,NaN,NaN,NaN


In [6]:
disturbances.columns

Index(['Table B.2.', 'Major Disturbances and Unusual Occurrences, 2002',
       'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7'],
      dtype='object')

Initial notes about data:
1. the column headers seem to be located in row 1
2. date formats are inconsistent
3. I will need to check what NERC Region means
4. What does area refer to? Sometimes it is an entire state while other times it is a more specific part of a state.
5. lots of NaN that needs to be addressed
6. some columns have inconsistent data located in them - perhaps these rows need to be removed

In [7]:
#What is the shape of the data:

disturbances.shape

(41, 8)

## Step 1. Creating a rough data cleaning plan in pseudocode 

After taking an initial look at the data I know I want to use the python package openpyxl to clean/ wrangle the data programmatically. Here is what I want to accomplish:
0. Load workbook so that we can read the data from the excel sheet with openpyxl
1. Initialise an empty pandas DataFrame. Define the column names: ["Date", "NERC Region", "Time", "Area Affected",	"Type of Disturbance", "Loss (megawatts)",	"Number of Customers Affected", "Restoration Date/Time"]
2. for sheets 2002 - 2010 we have columns A-H (in these sheets date and time of restoration are in one column called "Restoration Date/Time" which changes to two columns in 2011: "Date of Restoration", "Time of Restoration"):
    a. Select one sheet within the excel spreadsheet. loop rough the sheet line by line
    b. If row starts with "Table", remove row (or skip row)
    c. If row starts with a month or the word "Date", skip row
    d. else, append row to the DataFrame
2. for 2011-2014 we have columns A-I (look at data description to find out the new structure of the timing record). In 2011 "Type of disturbance" changes to "Event type".
3. for sheets 2015 - 2023 we have columns A-K: two columns have been added: "Month" (which replaces the row markers within the sheets, and "Alert Criteria"
3. Repeat step 2 for all sheets in the file
4. Check the output, start proper exploratory data analysis

In [19]:
#Step 0: Exploring openpyxl
excel_workbook = openpyxl.load_workbook("DOE_Electric_Disturbance_Events.xlsx")

# This is an example of how to return data from a predefined sheet
print(excel_workbook["2003"]["F7"].value) 

1000


In [23]:
#slicing the data - getting all cells from a column. I could also use iter_rows or iter_cols
print(excel_workbook["2015"]["A"])

(<Cell '2015'.A1>, <Cell '2015'.A2>, <Cell '2015'.A3>, <Cell '2015'.A4>, <Cell '2015'.A5>, <Cell '2015'.A6>, <Cell '2015'.A7>, <Cell '2015'.A8>, <Cell '2015'.A9>, <Cell '2015'.A10>, <Cell '2015'.A11>, <Cell '2015'.A12>, <Cell '2015'.A13>, <Cell '2015'.A14>, <Cell '2015'.A15>, <Cell '2015'.A16>, <Cell '2015'.A17>, <Cell '2015'.A18>, <Cell '2015'.A19>, <Cell '2015'.A20>, <Cell '2015'.A21>, <Cell '2015'.A22>, <Cell '2015'.A23>, <Cell '2015'.A24>, <Cell '2015'.A25>, <Cell '2015'.A26>, <Cell '2015'.A27>, <Cell '2015'.A28>, <Cell '2015'.A29>, <Cell '2015'.A30>, <Cell '2015'.A31>, <Cell '2015'.A32>, <Cell '2015'.A33>, <Cell '2015'.A34>, <Cell '2015'.A35>, <Cell '2015'.A36>, <Cell '2015'.A37>, <Cell '2015'.A38>, <Cell '2015'.A39>, <Cell '2015'.A40>, <Cell '2015'.A41>, <Cell '2015'.A42>, <Cell '2015'.A43>, <Cell '2015'.A44>, <Cell '2015'.A45>, <Cell '2015'.A46>, <Cell '2015'.A47>, <Cell '2015'.A48>, <Cell '2015'.A49>, <Cell '2015'.A50>, <Cell '2015'.A51>, <Cell '2015'.A52>, <Cell '2015'.A53>, <

In [29]:
row_dict = []
for row in excel_workbook["2015"].iter_rows(min_row=1, max_row=2, min_col=1, max_col=11, values_only=True):
    row_dict.append(row)

print(row_dict)

[('OE-417 Electric Emergency and Disturbance Report - Calendar Year 2015', None, None, None, None, None, None, None, None, None, None), ('Month', 'Date Event Began', 'Time Event Began', 'Date of Restoration', 'Time of Restoration', 'Area Affected', 'NERC Region', 'Alert Criteria', 'Event Type', 'Demand Loss (MW)', 'Number of Customers Affected')]


In [30]:
for row in excel_workbook["2015"].rows:
    print(row)

(<Cell '2015'.A1>, <MergedCell '2015'.B1>, <MergedCell '2015'.C1>, <MergedCell '2015'.D1>, <MergedCell '2015'.E1>, <MergedCell '2015'.F1>, <MergedCell '2015'.G1>, <MergedCell '2015'.H1>, <MergedCell '2015'.I1>, <MergedCell '2015'.J1>, <MergedCell '2015'.K1>, <Cell '2015'.L1>)
(<Cell '2015'.A2>, <Cell '2015'.B2>, <Cell '2015'.C2>, <Cell '2015'.D2>, <Cell '2015'.E2>, <Cell '2015'.F2>, <Cell '2015'.G2>, <Cell '2015'.H2>, <Cell '2015'.I2>, <Cell '2015'.J2>, <Cell '2015'.K2>, <Cell '2015'.L2>)
(<Cell '2015'.A3>, <Cell '2015'.B3>, <Cell '2015'.C3>, <Cell '2015'.D3>, <Cell '2015'.E3>, <Cell '2015'.F3>, <Cell '2015'.G3>, <Cell '2015'.H3>, <Cell '2015'.I3>, <Cell '2015'.J3>, <Cell '2015'.K3>, <Cell '2015'.L3>)
(<Cell '2015'.A4>, <Cell '2015'.B4>, <Cell '2015'.C4>, <Cell '2015'.D4>, <Cell '2015'.E4>, <Cell '2015'.F4>, <Cell '2015'.G4>, <Cell '2015'.H4>, <Cell '2015'.I4>, <Cell '2015'.J4>, <Cell '2015'.K4>, <Cell '2015'.L4>)
(<Cell '2015'.A5>, <Cell '2015'.B5>, <Cell '2015'.C5>, <Cell '2015'.D5>,

In [31]:
for index,row in disturbances.iterrows():
    print(row)

Table B.2.                                          NaN
Major Disturbances and Unusual Occurrences, 2002    NaN
Unnamed: 2                                          NaN
Unnamed: 3                                          NaN
Unnamed: 4                                          NaN
Unnamed: 5                                          NaN
Unnamed: 6                                          NaN
Unnamed: 7                                          NaN
Name: 0, dtype: object
Table B.2.                                                                  Date
Major Disturbances and Unusual Occurrences, 2002                     NERC Region
Unnamed: 2                                                                  Time
Unnamed: 3                                                                  Area
Unnamed: 4                                                   Type of Disturbance
Unnamed: 5                                                      Loss (megawatts)
Unnamed: 6                                 

In [11]:
#Step 1: Load workbook so that we can read the data from the excel sheet with openpyxl. Retrieve the sheet names.

# FYI There are additional reading options to keep in mind: read_only loads a spreadsheet in read-only mode allowing you to open very large Excel files.
# data_only ignores loading formulas and instead loads only the resulting values.

from openpyxl import load_workbook
excel_workbook = load_workbook(filename='DOE_Electric_Disturbance_Events.xlsx')
excel_sheetnames = excel_workbook.sheetnames
sheet = excel_workbook.active

1000
